In [1]:
import json
import pandas as pd
import numpy as np
from pandas import DataFrame

In [2]:
with open('purchase_data.json') as json_data:
    data = json.load(json_data)

In [12]:
#Create dataframe from data being read from the json file
df = pd.DataFrame.from_dict(data)
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [13]:
#Player count
total_rows = df[['Gender','SN','Age']].drop_duplicates()
total_players = total_rows.count()[0]
df_total_players = DataFrame({'Total Players': [total_players]})
df_total_players

,Total Players
0,573


In [29]:
#Purchasing Analysis (Total)
unique_items = df['Item ID'].nunique()
avg_purchase_price = df['Price'].mean()
total_purchases = df['Price'] .count()
total_revenue = df['Price'].sum()

df_purchasing = DataFrame({
    'Number of Unique Items':[unique_items],
    'Average Purchase Price':[avg_purchase_price],
    'Total Number of Purchases':[total_purchases],
    'Total Revenue':[total_revenue]
})
df_purchasing

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,2.931192,183,780,2286.33


In [30]:
#Gender Demographics
gender = total_rows['Gender'].value_counts()
male = gender[0]
female = gender[1]
other = gender[2]
male_pct = (male/total_players) *100
female_pct = (female/total_players) *100
other_pct = (other/total_players) *100

df_gender = pd.Series(gender).to_frame()
df_gender['Percentage'] = [male_pct.round(2),female_pct.round(2),other_pct.round(2)]
df_gender.rename(columns={'Gender':'Total Count'})

,Total Count,Percentage
Male,465,81.15
Female,100,17.45
Other / Non-Disclosed,8,1.40


In [23]:
#Purchasing Analysis (Gender)
grouped_by_gender = df.groupby('Gender')
purchase_by_gender = grouped_by_gender['Item ID'].count()
total_rev_gender = grouped_by_gender['Price'].sum()
normalized_totals = grouped_by_gender['Price'].mean()
avg_by_gender = grouped_by_gender['Price'].mean()
df_purchasing_gender = purchase_by_gender.to_frame()
df_purchasing_gender['Average Purchase Price',] = avg_by_gender.round(2)
df_purchasing_gender['Total Revenue']=total_rev_gender
df_purchasing_gender['Normalized Total']=normalized_totals.round(2)
df_purchasing_gender.rename(columns={'Item ID':'Purchase Count'})

,Purchase Count,Average Purchase Price,Total Revenue,Normalized Total
Gender,,,,
Female,136,2.82,382.91,2.82
Male,633,2.95,1867.68,2.95
Other / Non-Disclosed,11,3.25,35.74,3.25


In [24]:
#Age Demographics
bins = [0,10,15,20,25,30,35,40,45]
group_name = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
df['Age Bucket'] = pd.cut(df['Age'],bins, labels=group_name)
grouped_by_age = df.groupby('Age Bucket')
purchase_by_age = grouped_by_age['Item ID'].count()
total_rev_by_age = grouped_by_age['Price'].sum()
avg_by_age = grouped_by_age['Price'].mean()
normalized_total = grouped_by_age['Price'].mean()
df_age = purchase_by_age.to_frame()
df_age['Total Revenue'] = total_rev_by_age
df_age['Average Price'] = avg_by_age.round(2)
df_age['Normalized Total']=normalized_total.round(2)
df_age.rename(columns={'Item ID':'Purchase Count'})

,Purchase Count,Total Revenue,Average Price,Normalized Total
Age Bucket,,,,
<10,32,96.62,3.02,3.02
10-14,78,224.15,2.87,2.87
15-19,184,528.74,2.87,2.87
20-24,305,902.61,2.96,2.96
25-29,76,219.82,2.89,2.89
30-34,58,178.26,3.07,3.07
35-39,44,127.49,2.90,2.90
40+,3,8.64,2.88,2.88


In [36]:
#Top Spenders
group_spenders = df.groupby('SN')['Price'].sum()
top_spenders = group_spenders.sort_values(ascending=False)
top_spenders = top_spenders.iloc[:5]
top_spenders

SN
Undirrala66    17.06
Saedue76       13.56
Mindimnya67    12.74
Haellysu29     12.73
Eoda93         11.58
Name: Price, dtype: float64

In [26]:
#Most Popular Items
group_items = df.groupby('Item Name')['Item Name'].count()
top_items = group_items.sort_values(ascending=False)
top_items.iloc[:5]

Item Name
Final Critic                            14
Betrayal, Whisper of Grieving Widows    11
Arcane Gem                              11
Stormcaller                             10
Woeful Adamantite Claymore               9
Name: Item Name, dtype: int64

In [27]:
#Most Profitable Items
group_prices = df.groupby('Item Name')['Price'].sum()
most_prof = group_prices.sort_values(ascending=False)
most_prof.iloc[:5]

Item Name
Final Critic                  38.60
Retribution Axe               37.26
Stormcaller                   34.65
Spectral Diamond Doomblade    29.75
Orenmir                       29.70
Name: Price, dtype: float64